## Using QTL and GWAS struct to read files and filter for potential ivs.

1) imports

To enable multiple threads, create a new jupiter kernel with N threads ;

```julia
using IJulia
IJulia.installkernel("Julia 160 Threads", env=Dict(
    "JULIA_NUM_THREADS" => "160",
))
```
Then close and re-open the vscode window to restart the jupyter extention. It sould then detect the new kernels. 
Select the new kernel on the top right corner of the notebook view in vscode

In [1]:
using DLMReader
using Chain
using InMemoryDatasets
using Random
import Base.Threads.@threads
using StatsBase
using Distributions
using GLM
using Glob

2) Create QTl and GWAS objects

In [2]:
include("../src/inputs.jl")

In [12]:
file_pat = [TRAIT_NAME, "/exposure", TRAIT_NAME, "_chr", CHR, ".txt"]
folder = "/home/samat33/miniconda3/envs/julia_r/share/julia/dev/MrPainter/test/data/exposure"
trait_v = ["A", "C"]
chr_v = [1, 3]
tss_v = [398576, 9874365]
columns = Dict(1=>TRAIT_NAME, 2=>"etc...")
sep = ' '

' ': ASCII/Unicode U+0020 (category Zs: Separator, space)

In [13]:
QTLStudy_from_pattern(folder, file_pat, trait_v, chr_v, tss_v, columns, ' ')

QTLStudy(String[], Any[], ["A", "C"], [1, 3], [398576, 9874365], Dict{Int64, Any}(2 => "etc...", 1 => TRAIT_NAME), ' ')

real data

In [5]:
GWAS_file_path = "/mnt/r6_data/GWAS_MRsuite_hg38/input_MRsuite_BMI_Yengo_hg38"
QTL_prots_path = "/home/matpa01/CrickData_matpa01/Decode_mapping_suite"
QTL_prots_tss_file = "/mnt/r6_data/Patrick/Input_Decode/decode_list_hg38"

tss_df = filereader(QTL_prots_tss_file, delimiter = '_', header=[:trait, :chr, :tss], eolwarn = false)

dropmissing!(tss_df)

┌ Warning: There are problems with parsing the input file at line 723 (observation 723) : 
│ Column 2 : chr::Int64 : Read from buffer ("Y")
│  the values are set as missing.
│ MORE DETAILS: 
│ trait::String = CDY1.7097.8, chr::Int64 = missing, tss::Int64 = 25622162
│ CDY1.7097.8_Y_25622162
└ @ DLMReader ~/.julia/packages/DLMReader/N5BLJ/src/util.jl:952
┌ Warning: There are problems with parsing the input file at line 1295 (observation 1295) : 
│ Column 2 : chr::Int64 : Read from buffer ("Y")
│  the values are set as missing.
│ MORE DETAILS: 
│ trait::String = EIF1AY.19224.5, chr::Int64 = missing, tss::Int64 = 20575776
│ EIF1AY.19224.5_Y_20575776
└ @ DLMReader ~/.julia/packages/DLMReader/N5BLJ/src/util.jl:952


,trait,chr,tss
,identity,identity,identity
,String?,Int64?,Int64?
1,A1BG.16561.9,19,58353492
2,A1CF.12423.38,10,50885675
3,A2ML1.15482.12,12,8822621
4,A2M.3708.62,12,9116229
5,A4GALT.8759.29,22,42721298
6,A4GNT.10835.25,3,138132390
7,AAGAB.11926.23,15,67255195
8,AASDHPPT.17320.19,11,106075501


In [14]:
Decode_pat = [TRAIT_NAME, "/", TRAIT_NAME, "_chromosome_", CHR, ".tsv"]
Decode_columns = Dict(1 => "id", 2 => CHR, 3 => POS, 4 => A_EFFECT, 5 => A_OTHER, 6 => BETA, 7 => SE, 8 => PVAL)
sep = '\t'

Decode = QTLStudy_from_pattern(QTL_prots_path,Decode_pat, tss_df.trait, tss_df.chr, tss_df.tss, Decode_columns, sep, true)

QTLStudy(["/home/matpa01/CrickData_matpa01/Decode_mapping_suite/A1BG.16561.9/A1BG.16561.9_chromosome_19.tsv", "/home/matpa01/CrickData_matpa01/Decode_mapping_suite/A1CF.12423.38/A1CF.12423.38_chromosome_10.tsv", "/home/matpa01/CrickData_matpa01/Decode_mapping_suite/A2M.3708.62/A2M.3708.62_chromosome_12.tsv", "/home/matpa01/CrickData_matpa01/Decode_mapping_suite/A2ML1.15482.12/A2ML1.15482.12_chromosome_12.tsv", "/home/matpa01/CrickData_matpa01/Decode_mapping_suite/A4GALT.8759.29/A4GALT.8759.29_chromosome_22.tsv", "/home/matpa01/CrickData_matpa01/Decode_mapping_suite/A4GNT.10835.25/A4GNT.10835.25_chromosome_3.tsv", "/home/matpa01/CrickData_matpa01/Decode_mapping_suite/AAGAB.11926.23/AAGAB.11926.23_chromosome_15.tsv", "/home/matpa01/CrickData_matpa01/Decode_mapping_suite/AASDHPPT.17320.19/AASDHPPT.17320.19_chromosome_11.tsv", "/home/matpa01/CrickData_matpa01/Decode_mapping_suite/ABCC6.8935.22/ABCC6.8935.22_chromosome_16.tsv", "/home/matpa01/CrickData_matpa01/Decode_mapping_suite/ABHD10.17

In [17]:
Decode.tss_v

4483-element Vector{Union{Missing, Int64}}:
  58353492
  50885675
   8822621
   9116229
  42721298
 138132390
  67255195
 106075501
  16223522
 111979010
  25390835
  51983409
 130713016
         ⋮
  36489902
  90352284
  31443160
  28883572
  28883572
 237890922
  39868202
  50121329
 102099244
  66504959
  56361273
 143381295